In [1]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd

def setup_driver():
    """Initialize Selenium WebDriver."""
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run browser in headless mode
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    driver_path = r"C:\Users\Sajid\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"  # Replace with your ChromeDriver path
    service = Service(driver_path)
    return webdriver.Chrome(service=service, options=chrome_options)



In [2]:
def scrape_pinterest_images(search_query, num_images=20):
    """
    Scrape Pinterest images based on a search query.
    
    Args:
        search_query (str): The search term for Pinterest.
        num_images (int): Number of images to scrape.
    
    Returns:
        list: List of dictionaries containing image URLs and filenames.
    """
    # Create URL for Pinterest search
    search_url = f"https://www.pinterest.com/search/pins/?q={search_query.replace(' ', '%20')}"
    driver = setup_driver()
    driver.get(search_url)

    # Scroll and load images dynamically
    image_urls = set()
    scroll_pause_time = 2
    scroll_count = 0

    while len(image_urls) < num_images and scroll_count < 10:
        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        
        # Find all image elements
        image_elements = driver.find_elements(By.CSS_SELECTOR, "img")
        for img in image_elements:
            src = img.get_attribute("src")
            if src and "pinimg.com" in src:
                image_urls.add(src)
        
        scroll_count += 1
    
    # Quit driver
    driver.quit()

    # Download images
    folder_name = "pinterest_images"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    
    data = []
    for i, url in enumerate(list(image_urls)[:num_images]):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                filename = f"{folder_name}/image_{i + 1}.jpg"
                with open(filename, "wb") as f:
                    f.write(response.content)
                data.append({"Image URL": url, "Filename": filename})
                print(f"Downloaded: {filename}")
        except Exception as e:
            print(f"Error downloading image {url}: {e}")
    
    return data

def save_to_csv(data, filename="pinterest_images.csv"):
    """
    Save image data to a CSV file.
    
    Args:
        data (list): List of dictionaries containing image data.
        filename (str): Name of the output CSV file.
    """
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False, encoding="utf-8")
    print(f"Data saved to {filename}")

if __name__ == "__main__":
    search_term = "nature wallpaper"  # Replace with your search term
    num_images_to_download = 20  # Specify the number of images to scrape
    scraped_data = scrape_pinterest_images(search_term, num_images_to_download)
    save_to_csv(scraped_data)


Downloaded: pinterest_images/image_1.jpg
Downloaded: pinterest_images/image_2.jpg
Downloaded: pinterest_images/image_3.jpg
Downloaded: pinterest_images/image_4.jpg
Downloaded: pinterest_images/image_5.jpg
Downloaded: pinterest_images/image_6.jpg
Downloaded: pinterest_images/image_7.jpg
Downloaded: pinterest_images/image_8.jpg
Downloaded: pinterest_images/image_9.jpg
Downloaded: pinterest_images/image_10.jpg
Downloaded: pinterest_images/image_11.jpg
Downloaded: pinterest_images/image_12.jpg
Downloaded: pinterest_images/image_13.jpg
Downloaded: pinterest_images/image_14.jpg
Downloaded: pinterest_images/image_15.jpg
Downloaded: pinterest_images/image_16.jpg
Downloaded: pinterest_images/image_17.jpg
Downloaded: pinterest_images/image_18.jpg
Downloaded: pinterest_images/image_19.jpg
Downloaded: pinterest_images/image_20.jpg
Data saved to pinterest_images.csv
